# Router Chains: Selecting from multiple prompts with MultiRetrievalQAChain

This notebook demonstrates how to use the `RouterChain` paradigm to create a chain that dynamically selects which Retrieval system to use. Specifically we show how to use the `MultiRetrievalQAChain` to create a question-answering chain that selects the retrieval QA chain which is most relevant for a given question, and then answers the question using it.

In [1]:
from langchain.chains.router import MultiRetrievalQAChain
from langchain.llms import OpenAI

In [2]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS

sou_docs = TextLoader('../../state_of_the_union.txt').load_and_split()
sou_retriever = FAISS.from_documents(sou_docs, OpenAIEmbeddings()).as_retriever()

pg_docs = TextLoader('../../paul_graham_essay.txt').load_and_split()
pg_retriever = FAISS.from_documents(pg_docs, OpenAIEmbeddings()).as_retriever()

personal_texts = [
    "I love apple pie",
    "My favorite color is fuchsia",
    "My dream is to become a professional dancer",
    "I broke my arm when I was 12",
    "My parents are from Peru",
]
personal_retriever = FAISS.from_texts(personal_texts, OpenAIEmbeddings()).as_retriever()


In [3]:
retriever_infos = [
    ("state of the union", "Good for answering questions about the 2023 State of the Union address", sou_retriever),
    ("pg essay", "Good for answer quesitons about Paul Graham's essay on his career", pg_retriever),
    ("personal", "Good for answering questions about me", personal_retriever)
]
chain = MultiRetrievalQAChain.from_retrievers(OpenAI(), *zip(*retriever_infos), verbose=True)

In [4]:
print(chain.run("What did the president say about the economy?"))



> Entering new MultiRetrievalQAChain chain...
state of the union: {'query': 'What did the president say about the economy in the 2023 State of the Union address?'}
> Finished chain.
 The president said that the economy was growing, that it had created more jobs in one year than ever before in the history of America, and that it had the strongest growth in nearly 40 years. He also discussed his economic vision for America, his plan to fight inflation, and his plan to lower costs for families.


In [7]:
print(chain.run("What is something Paul Graham regrets about his work?"))



> Entering new MultiRetrievalQAChain chain...
pg essay: {'query': 'What is something Paul Graham regrets about his work?'}
> Finished chain.
 Paul Graham regrets that working on Hacker News was the biggest source of stress in his work.


In [8]:
print(chain.run("What is my background?"))



> Entering new MultiRetrievalQAChain chain...
personal: {'query': 'What is my background?'}
> Finished chain.
 Your background is Peruvian.


In [9]:
print(chain.run("What year was the Internet created in?"))



> Entering new MultiRetrievalQAChain chain...
DEFAULT: {'query': 'What year was the Internet created in?'}
> Finished chain.
The Internet was created in the late 1960s. However, it wasn't until the 1980s that it became widely available for public use. The first successful message transmission over the Internet occurred in October 1969 between two computers located at UCLA and the Stanford Research Institute. The development of the World Wide Web in the early 1990s helped to popularize the Internet and make it more accessible to the general public.
